In [1]:
import numpy as np

## 1.Modified Duration : rủi ro của trái phiếu

In [1]:
def durations_explicit(c, y, m, n):
    '''
    c: coupon
    y: yiel
    m: tần suất trả lãi
    n = số năm

    '''
    macaulay_duration = ((1+y) / (m*y)) - ( (1 + y + n*(c-y)) / ((m*c* ((1+y)**n - 1)) + m*y) )
    modified_duration = macaulay_duration / (1 + y)
    return macaulay_duration, modified_duration

durations_explicit(c=0.03,y=0.08,m=2,n=3)

(1.4533512181608765, 1.3456955723711819)

In [3]:
# Câu trả lời đầy đủ hơn các tại liệu cho trường hợp tỷ lệ trái phiếu giảm giá mỗi kì bằng với năng suất mỗi kì
def durations_explicit(c, y, m, n):
    """Parameters:
        c = coupon rate per period
        y = yield per period
        m = periods per year
        n = periods remaining"""
    if c==y: # Shorter explicit formula if coupon rate per period = yield per period
        macaulay_duration = ((1+y)/(m*y))*(1 - (1 / (1+y)**n))
        modified_duration = macaulay_duration / (1 + y)
        print(f"Macaulay Duration: {macaulay_duration}")
        print(f"Modified Duration: {modified_duration}")
        return macaulay_duration, modified_duration
    else:
        macaulay_duration = ((1+y) / (m*y)) - ( (1 + y + n*(c-y)) / ((m*c* ((1+y)**n - 1)) + m*y) )
        modified_duration = macaulay_duration / (1 + y)
        print(f"Macaulay Duration: {macaulay_duration}")
        print(f"Modified Duration: {modified_duration}")
        return macaulay_duration, modified_duration

durations_explicit(c=0.06, y=0.08, m=1, n=3)
# Nếu lãi suất tăng 1% 1 năm thì trái phiếu sẽ giảm khoảng 2.16% trong 1 năm hoặc ngược lại nếu lãi giảm 1%


Macaulay Duration: 2.8286150467364823
Modified Duration: 2.6190880062374835


(2.8286150467364823, 2.6190880062374835)

## price change with conversity and duration



In [5]:
import pandas as pd
import numpy as np
def price_change(rate,coupon_rate, frequency, face_value,settlement_date, maturity_date, change_in_year):
    try:
        settlement_date = pd.to_datetime(settlement_date, format="%d/%m/%Y")
    except:
        settlement_date = pd.to_datetime(settlement_date, format="%d-%m-%Y")

    try:
        maturity_date = pd.to_datetime(maturity_date, format="%d/%m/%Y")
    except:
        maturity_date = pd.to_datetime(maturity_date, format="%d-%m-%Y")

    data = pd.DataFrame()
    rate = rate/100
    coupon_rate = coupon_rate/100

    n =pd.to_numeric(((pd.to_datetime(maturity_date) - pd.to_datetime(settlement_date))/365).days)
    total_payment = n*frequency
    coupon_payment = coupon_rate/frequency * face_value
    payment = [coupon_payment]*(total_payment-1)+[coupon_payment+face_value]
    data['period'] = pd.DataFrame(np.arange(1,total_payment+1))
    data['payment'] = pd.DataFrame(payment)
    data['dcoupon'] = data['payment']/((1+ rate/frequency)**data['period'])
    data['pv'] = data['dcoupon'] / frequency * data['period'] / data['dcoupon'].sum()
    duration = data['pv'].sum()
    m_duration = (duration/(1+rate/frequency))

    # convexity calculation 

    factor = 1/ (data['dcoupon'].sum()*(1+rate/frequency)**2)
    data['cf'] = data['dcoupon']*(data['period']**2 + data['period'])/ (1+rate/frequency)**data['period']
    convexity = factor * data['cf'].sum()

    # price change by using duration and converxity
    # price change = Duration Effect + converxity
    price_change = (-duration*change_in_year*100)+(0.5*convexity*(change_in_year)**2*100)


    return duration, m_duration, convexity,price_change,data

In [8]:
price_change(7,3.915,2,100,'15-05-2019','10-3-2034',0.01)

(10.265732191600454,
 9.91858182763329,
 203.1023511842184,
 -9.250220435679363,
     period   payment    dcoupon        pv            cf
 0        1    1.9575   1.891304  0.012999      3.654694
 1        2    1.9575   1.827347  0.025119     10.235089
 2        3    1.9575   1.765553  0.036404     19.109130
 3        4    1.9575   1.705848  0.046897     29.730962
 4        5    1.9575   1.648162  0.056639     41.631257
 5        6    1.9575   1.592428  0.065668     54.408514
 6        7    1.9575   1.538577  0.074022     67.721240
 7        8    1.9575   1.486548  0.081736     81.280931
 8        9    1.9575   1.436278  0.088844     94.845774
 9       10    1.9575   1.387709  0.095377    108.214999
 10      11    1.9575   1.340781  0.101367    121.223831
 11      12    1.9575   1.295441  0.106843    133.738970
 12      13    1.9575   1.251634  0.111832    145.654553
 13      14    1.9575   1.209308  0.116362    156.888558
 14      15    1.9575   1.168413  0.120457    167.379597
 15    